# Impacket Lateralization Detection
Detects wmiexec/dcomexec/atexec/smbexec from Impacket framework

## Rule Content
```
- title: Impacket Lateralization Detection
  id: 10c14723-61c7-4c75-92ca-9af245723ad2
  status: experimental
  description: Detects wmiexec/dcomexec/atexec/smbexec from Impacket framework
  references:
  - https://github.com/SecureAuthCorp/impacket/blob/master/examples/wmiexec.py
  - https://github.com/SecureAuthCorp/impacket/blob/master/examples/atexec.py
  - https://github.com/SecureAuthCorp/impacket/blob/master/examples/smbexec.py
  - https://github.com/SecureAuthCorp/impacket/blob/master/examples/dcomexec.py
  author: Ecco
  date: 2019/09/03
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection_other:
      ParentImage:
      - '*\wmiprvse.exe'
      - '*\mmc.exe'
      - '*\explorer.exe'
      - '*\services.exe'
      CommandLine:
      - '*cmd.exe* /Q /c * \\\\127.0.0.1\\*&1*'
    selection_atexec:
      ParentCommandLine:
      - '*svchost.exe -k netsvcs'
      - taskeng.exe*
      CommandLine:
      - cmd.exe /C *Windows\\Temp\\*&1
    condition: (1 of selection_*)
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.lateral_movement
  - attack.t1047
  - attack.t1175
  falsepositives:
  - pentesters
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:(*\\wmiprvse.exe OR *\\mmc.exe OR *\\explorer.exe OR *\\services.exe) AND process_command_line.keyword:(*cmd.exe*\ \/Q\ \/c\ *\ \\\\127.0.0.1\\*&1*)) OR (process_parent_command_line.keyword:(*svchost.exe\ \-k\ netsvcs OR taskeng.exe*) AND process_command_line.keyword:(cmd.exe\ \/C\ *Windows\\Temp\\*&1)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()